In [11]:
import numpy
import pickle as pk
import os
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [2]:
os.chdir('/home/jcai/geometry_of_law/data_and_dictionary')

In [3]:
master_dataframe = pd.read_csv("/home/jcai/geometry_of_law/Encyclopedia Entry/master_dataframe.csv")

In [4]:
#loading
#Y_average_dict = pk.load(open( "Y_average_dict.p", "rb" ))
#CBY_average_dict = pk.load(open( "CBY_average_dict.p", "rb" ))
#JY_average_dict = pk.load(open( "JY_average_dict.p", "rb" ))
#YB_average_dict = pk.load(open( "YB_average_dict.p", "rb" ))
CB_average_dict = pk.load(open( "CB_average_dict.p", "rb" ))
#CY_average_dict = pk.load(open( "CY_average_dict.p", "rb" ))
#JD_average_dict = pk.load(open( "JD_average_dict.p", "rb" ))
#CD_average_dict = pk.load(open( "CD_average_dict.p", "rb" ))
#C_average_dict = pk.load(open( "C_average_dict.p", "rb" ))
#judge_average_dict = pk.load(open( "judge_average_dict.p", "rb" ))

In [5]:
#Y_dict = pk.load(open( "Y_dict.p", "rb" ))
#CBY_dict = pk.load(open( "CBY_dict.p", "rb" ))
#JY_dict = pk.load(open( "JY_dict.p", "rb" ))
#YB_dict = pk.load(open( "YB_dict.p", "rb" ))
CB_dict = pk.load(open( "CB_dict.p", "rb" ))
#CY_dict = pk.load(open( "CY_dict.p", "rb" ))
#JD_dict = pk.load(open( "JD_dict.p", "rb" ))
#CD_dict = pk.load(open( "CD_dict.p", "rb" ))
#C_dict = pk.load(open( "C_dict.p", "rb" ))
docname_vector_dict = pk.load(open( "docname_vector_dict.p", "rb" ))
#judge_docname_list_dict = pk.load(open( "judge_docname_list_dict.p", "rb" ))

In [21]:
#helper
def return_average_vector(list_of_docname):
    '''returns the average vector for the given list of docname'''
    list_of_vectors = [docname_vector_dict[x] for x in list_of_docname]
    mean = np.mean(list_of_vectors, axis=0)
    return mean

def return_average_demeaned_vector(list_of_docname,mean_dict,item_dict):
    '''returns the average demeaned vector for the given list of docname'''
    '''list_dict: item to list; mean_dict: item to mean vector; returns a demeaned dictionary'''
    list_of_vectors = [docname_vector_dict[x] - mean_dict[item_dict[x]]  for x in list_of_docname]
    mean = np.mean(list_of_vectors, axis=0)
    return mean

def run_PCA_tSNE(X,PCA_components,TSNE_components):
    '''returns a data frame with 2 columns that has been PCA and tsned'''
    pca = PCA(n_components=PCA_components)
    pca_result = pca.fit_transform(X)
    #scaler = StandardScaler()
    #df_pca_result_fit_transform = scaler.fit_transform(pca_result)
    tsne = TSNE(n_components=TSNE_components) 
    tsne_results = tsne.fit_transform(pca_result) #used to be df_pca_result_fit_transform
    df_tsne = pd.DataFrame(tsne_results)
    return df_tsne

def return_list_of_docname(column, value):
    '''returns the list of docname of the vectors whose column is value'''
    list_of_docname = list(master_dataframe.loc[master_dataframe[column]==value]['docname'])
    return list_of_docname

In [18]:
#find the set of docnames (opinions) by judge-decade
list_of_judges = list(master_dataframe['judge_name'].unique())
list_list_of_docnames_judges = [return_list_of_docname(column = 'judge_name', value = x) for x in list_of_judges]

In [22]:
judge_data = pd.read_csv('judge_data.csv')
CB_demeaned_vectors_for_judges = [return_average_demeaned_vector(list_of_docname = x,mean_dict=CB_average_dict,item_dict=CB_dict) for x in list_list_of_docnames_judges]
#make a dictionary
CB_demeaned_vectors_for_judges_dict = dict(zip(list_of_judges, CB_demeaned_vectors_for_judges))
#save and export
X = pd.DataFrame.from_dict(CB_demeaned_vectors_for_judges_dict, orient='index')
#numpy.isnan(X1).any()
#numpy.isfinite(X1).all()
df_tsne = run_PCA_tSNE(X,50,2)
#rename columns
col_names = list(df_tsne.columns)
col_names[0] = 'x'
col_names[1] = 'y'
df_tsne.columns = col_names
#here we merge and save dataframes
CB_demeaned_vectors_for_judges = pd.merge(df_tsne, judge_data,left_index=True, right_index=True)
CB_demeaned_vectors_for_judges.to_csv('CB_demeaned_vectors_for_judges.csv',index=False)